# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [21]:
import pandas as pd

In [22]:
df = pd.read_csv('./clean_data_after_eda.csv')

In [23]:
df_copy=df.copy()

In [24]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              14606 non-null  object 
 1   channel_sales                   14606 non-null  object 
 2   cons_12m                        14606 non-null  int64  
 3   cons_gas_12m                    14606 non-null  int64  
 4   cons_last_month                 14606 non-null  int64  
 5   date_activ                      14606 non-null  object 
 6   date_end                        14606 non-null  object 
 7   date_modif_prod                 14606 non-null  object 
 8   date_renewal                    14606 non-null  object 
 9   forecast_cons_12m               14606 non-null  float64
 10  forecast_cons_year              14606 non-null  int64  
 11  forecast_discount_energy        14606 non-null  float64
 12  forecast_meter_rent_12m         

---
## 2. Load data

In [25]:
df_copy["date_activ"] = pd.to_datetime(df_copy["date_activ"], format='%Y-%m-%d')
df_copy["date_end"] = pd.to_datetime(df_copy["date_end"], format='%Y-%m-%d')
df_copy["date_modif_prod"] = pd.to_datetime(df_copy["date_modif_prod"], format='%Y-%m-%d')
df_copy["date_renewal"] = pd.to_datetime(df_copy["date_renewal"], format='%Y-%m-%d')

In [26]:
df_copy.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [27]:
# price_df = pd.read_csv('price_data.csv')
# price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
# price_df.head()

In [28]:
# # Group off-peak prices by companies and month
# monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# # Get january and december prices
# jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
# dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# # Calculate the difference
# diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
# diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
# diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
# diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
# diff.head()

### making new features from existing ones like tenure,forecasted price difference, ratio of net and gross margin and new consumption feature

### Tenure

In [29]:
df_copy['tenure'] = (df_copy['date_end'] - df_copy['date_activ']).dt.days //365.0

without dt.days we get timedelta object as ,1096 days , like that so to convert it into int64 we need timedelta accessor i.e .dt.days or .dt.seconds or more 

In [30]:
df_copy['tenure'].head()

0    3.0
1    7.0
2    6.0
3    6.0
4    6.0
Name: tenure, dtype: float64

### Forcasted price difference

In [31]:
df_copy['forecast_price_diff'] = df_copy['forecast_price_energy_peak'] - df_copy['forecast_price_energy_off_peak']

In [32]:
import numpy as np
# np.where()

### margin ratio

In [33]:
df_copy['margin_ratio'] = np.where(df_copy['margin_gross_pow_ele'] != 0, df_copy['margin_net_pow_ele'] / df_copy['margin_gross_pow_ele'], 0)


### Monthly consumption ration

In [34]:
average_monthly_consumption = df_copy['cons_12m'] / 12.0
df_copy['consumption_ratio'] = np.where(average_monthly_consumption != 0,df_copy['cons_last_month'] / average_monthly_consumption,0)


### Margin pe kwh

In [35]:
df_copy["margin_per_kwh"] = np.where(df_copy["cons_12m"] == 0, 0,df_copy["margin_net_pow_ele"] / df_copy["cons_12m"])


In [36]:
df_copy["time_since_modif"] = (df_copy["date_end"] - df_copy["date_modif_prod"]).dt.days
df_copy["time_to_renewal"] = (df_copy["date_end"] - df_copy["date_renewal"]).dt.days

In [37]:
df_copy['has_gas'].unique()

array(['t', 'f'], dtype=object)

In [38]:
df_copy['has_gas']=df_copy['has_gas'].map({'t':1,'f':0}).astype("int64")

DROPPING COLUMNS WHICH WE HAVE USED TO CREATE NEW ONES

In [39]:
columns_to_drop = [
    'date_activ',
    'date_end',
    'forecast_price_energy_peak',
    'forecast_price_energy_off_peak',
    'margin_gross_pow_ele',
    'margin_net_pow_ele',
    'cons_12m',
    'cons_last_month',
    'date_modif_prod',
    'date_renewal',
    'origin_up',
    'id',
    'channel_sales'
    
]
df_copy.drop(columns=columns_to_drop,inplace=True)


In [40]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   cons_gas_12m                 14606 non-null  int64  
 1   forecast_cons_12m            14606 non-null  float64
 2   forecast_cons_year           14606 non-null  int64  
 3   forecast_discount_energy     14606 non-null  float64
 4   forecast_meter_rent_12m      14606 non-null  float64
 5   forecast_price_pow_off_peak  14606 non-null  float64
 6   has_gas                      14606 non-null  int64  
 7   imp_cons                     14606 non-null  float64
 8   nb_prod_act                  14606 non-null  int64  
 9   net_margin                   14606 non-null  float64
 10  num_years_antig              14606 non-null  int64  
 11  pow_max                      14606 non-null  float64
 12  var_year_price_off_peak_var  14606 non-null  float64
 13  var_year_price_p